<div style="text-align:center"><img src="assets/intro.jpg"/></div>

# <span style="color:#264896">Programme de détection de faux billets</span>

Pour lancer le script dans son intégralité et obtenir le résultat en fin de Notebook, veuillez cliquer sur le bouton suivant situé dans la **barre d'outils** ci-dessus :

![tuto](assets/tuto.jpg)

### <span style="color:#17aafa">Sommaire</span>
1. Création du modèle de prédiction
2. Import et contrôle du fichier à tester
3. Application de l'algorithme de prédiction
4. Résultats du traitement

In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection

### <span style="color:#17aafa" id="modele">Création du modèle de prédiction</span>

In [2]:
#importation du fichier d'entrainement
dataset = pd.read_csv("datas/notes.csv", sep=",", decimal=".", header=0)

#transformation de la variable is_genuine
#True = 0
#False = 1
dataset['is_genuine'] = dataset['is_genuine'].apply(lambda x: 0 if x == True else 1)
dataset["is_genuine"] = dataset["is_genuine"].astype('category')

# on définit x et y
y = dataset["is_genuine"].cat.codes
X = dataset.select_dtypes(np.number)

#subdivision des données en entrainement et test
X_train,X_test,y_train,y_test = model_selection.train_test_split (X,y,test_size = 0.2 ,random_state=0)

#regression logistique
lr = LogisticRegression()

#modélisation sur les données d'apprentissage
modele_sk = lr.fit(X_train, y_train)

### <span style="color:#17aafa">Import et contrôle du fichier à tester</span>

In [3]:
from tkinter import * 
from tkinter import filedialog 
from tkinter import messagebox

#sélection du fichier
root = Tk() 
root.filename = filedialog.askopenfilename(initialdir = "C:/Users/OpenClassrooms/jupyter-notebooks/P6_Détectez_des_faux_billets/",
                                           title = "Sélectionnez le fichier",
                                           filetypes = (("Fichiers CSV","*.csv"),("all files","*.*"))) 
root.withdraw()

dataset_new = pd.read_csv(root.filename, sep=",", decimal=".", header=0)

In [4]:
#vérification des variables
liste_X = ["diagonal","height_left","height_right","margin_low","margin_up","length","id"]
if all([item in dataset_new.columns for item in liste_X]):
    print("Les variables du fichier sont correctes. Voici un aperçu des données :\n\n",dataset_new)
else :
    messagebox.showerror(title="Erreur d'importation du fichier", message="Attention, les colonnes du fichier "+root.filename+" ne correspondent pas à la trame attendue !")
    sys.exit()

Les variables du fichier sont correctes. Voici un aperçu des données :

    diagonal  height_left  height_right  margin_low  margin_up  length   id
0    172.09       103.95        103.73        4.39       3.09  113.19  B_1
1    171.52       104.17        104.03        5.27       3.16  111.82  B_2
2    171.78       103.80        103.75        3.81       3.24  113.39  B_3
3    172.02       104.08        103.99        5.57       3.30  111.10  B_4
4    171.79       104.34        104.37        5.00       3.07  111.87  B_5


In [5]:
if((dataset_new.isnull().sum(axis = 0)).sum() > 0):
    messagebox.showerror(title="Valeurs nulles détectées", message="Attention, des valeurs nulles ont été détectées dans le fichier ! Traitement impossible en l'état.")
    sys.exit()
else :
    print("Aucune valeur nulle détectée, le traitement peut continuer.")

Aucune valeur nulle détectée, le traitement peut continuer.


### <span style="color:#17aafa">Application de l'algorithme de prédiction</span>

In [6]:
#creation des variables 
X_new = dataset_new[liste_X[:-1]]

#calcul des probabilités d'affectation sur l'échantillon exemple
probas_new = lr.predict_proba(X_new)

#score de presence : Probalilité de chance que le billet soit VRAI (is_genuine = 0)
score_new = probas_new[:,0]

#Ajout de la colonne probabilité
dataset_new["proba_true"] = score_new
#calcul de la variable is_genuine
dataset_new["is_genuine"] = np.where(dataset_new["proba_true"]>=0.5, 'Oui', 'Non')
#traitement du Dataframe pour meilleure compréhension :
dataset_new_fin = dataset_new[["id","proba_true","is_genuine"]].rename(columns={"id":"Identifiant du billet testé",
                                                                                 "proba_true":"Probabilité que le billet soit VRAI (%)",
                                                                                 "is_genuine":"Est authentique ?"})
dataset_new_fin["Probabilité que le billet soit VRAI (%)"] = round((dataset_new_fin["Probabilité que le billet soit VRAI (%)"]*100),2)

### <span style="color:#17aafa">Résultats du traitement</span>

Selon la consigne donnée, pour chaque billet, l'algorithme de classification devra donner la **probabilité que le billet soit vrai**. Si cette probabilité est supérieure ou égale à 0.5 *(50%)*, le billet sera considéré comme vrai. Dans le cas contraire, il sera considéré comme faux.

In [7]:
dataset_new_fin

,Identifiant du billet testé,Probabilité que le billet soit VRAI (%),Est authentique ?
0,B_1,95.37,Oui
1,B_2,7.22,Non
2,B_3,99.28,Oui
3,B_4,0.59,Non
4,B_5,10.35,Non


In [8]:
liste_faux_billets = list(dataset_new_fin.loc[(dataset_new_fin["Est authentique ?"] == "Non"),"Identifiant du billet testé"].values)

In [9]:
print("Les billets dont l'identifiant est dans la liste", liste_faux_billets, "\nont été identifiés commes FAUX selon l'algorithme de prédiction.")

Les billets dont l'identifiant est dans la liste ['B_2', 'B_4', 'B_5'] 
ont été identifiés commes FAUX selon l'algorithme de prédiction.
